In [2]:
#!/usr/bin/python3

import numpy as np
from sys import argv
import pyoma.browser.db as db
import pyoma.browser.models as mod
import zoo.wrappers.aligners.mafft as mafft
import zoo.wrappers.treebuilders.fasttree as fasttree
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Alphabet import IUPAC, SingleLetterAlphabet
from Bio.Seq import Seq, UnknownSeq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict

from os import listdir
from os.path import isfile, join
from datetime import datetime

import concurrent.futures


import ast
#  for development 
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt


datasets_address= "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/archive/"



oma_database_address = datasets_address + "OmaServer.h5"
hog_og_map_address = datasets_address + "hog_og_map.dic"


omaID_address = datasets_address+"oma-species.txt"
bird6ID_address = datasets_address+"info.tsv"


# should end in /

# very small
project_folder = "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/ST/f4_100S/" 
#project_folder = "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/" 

#project_folder = argv[1]


# PANPA.fa  PANPA.hogmap
# The species name of query is the name of the file; 
#  argv[2] 




In [ ]:

def parse_oma(oma_database_address, hog_og_map_address):
    
    ############### Parsing OMA db ####################
    ###################################################

    oma_db = db.Database(oma_database_address)

    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- OMA data is parsed and its release name is:", oma_db.get_release_name())
    list_speices= [z.uniprot_species_code for z in oma_db.tax.genomes.values()] 
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- There are",len(list_speices),"species in the OMA database.")

    file = open(hog_og_map_address, "r")
    contents = file.read()
    hog_OG_map = ast.literal_eval(contents)
    file.close()
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- The hog-og map is read from file with the length of ", len(hog_OG_map))
    
    
    return (oma_db, hog_OG_map, list_speices)


def parse_proteome(project_folder, list_speices):
    
    ############### Parsing query proteome of species #######
    #########################################################

    project_files = listdir(project_folder)

    query_species_names = []
    for file in project_files:
        if file.split(".")[-1]=="fa":
            file_name_split = file.split(".")[:-1]
            query_species_names.append('.'.join(file_name_split))

    # we may assert existence of query_species_name+".fa/hogmap"
    query_prot_records_species = [ ]
    for query_species_name in query_species_names:
        query_prot_address = project_folder + query_species_name + ".fa" 
        query_prot_records = list(SeqIO.parse(query_prot_address, "fasta")) 
        query_prot_records_species.append(query_prot_records)

    # for development
    query_species_num = len(query_species_names)
    for species_i in range(query_species_num):
        len_prot_record_i = len( query_prot_records_species[species_i] )
        species_name_i = query_species_names[species_i]
        #print(species_name_i,len_prot_record_i)
        if species_name_i in list_speices: 
            current_time = now.strftime("%H:%M:%S")
            print(current_time,"- the species",species_name_i," already exists in the oma database, remove it first")
            exit()

    return (query_species_names, query_prot_records_species)



def parse_hogmap_omamer(project_folder , query_species_names):

    ################### Parsing omamer's output  ########
    #####################################################

    query_prot_names_species = []
    query_hogids_species = []

    for query_species_name in query_species_names:
        omamer_output_address = project_folder + query_species_name + ".hogmap"     
        omamer_output_file = open(omamer_output_address,'r');

        query_prot_names= []
        query_hogids= []

        for line in omamer_output_file:
            line_strip=line.strip()
            if not line_strip.startswith('qs'):
                line_split= line_strip.split("\t")        
                query_prot_names.append(line_split[0])
                query_hogids.append(line_split[1])
        #print("number of proteins in omamer output for ",query_species_name,"is",len(query_hogids)) # ,query_hogids
        query_prot_names_species.append(query_prot_names)
        query_hogids_species.append(query_hogids)    
    
    return (query_prot_names_species, query_hogids_species)
    
    
    

def extract_unique_hog(query_species_names,query_hogids_species, query_prot_names_species,query_prot_records_species):
    ###### Extracting unique HOG list and corresponding query proteins ########
    ###########################################################################

    query_hogids_filtr_species = []
    query_prot_names_filtr_species = []
    query_prot_records_filtr_species = []

    repeated_hogid_num = 0
    
    query_species_num = len(query_species_names) 
    
    for species_i in range(query_species_num):
        #print(query_species_names[species_i])

        query_hogids =  query_hogids_species[species_i]
        query_prot_names = query_prot_names_species[species_i]
        query_prot_records  = query_prot_records_species[species_i]



        query_hogids_filtr = []
        query_prot_names_filtr = []
        query_prot_records_filtr = []

        for prot_i in range(len(query_hogids)):

            if not query_hogids[prot_i] in query_hogids_filtr: 

                query_hogids_filtr.append(query_hogids[prot_i])
                query_prot_names_filtr.append(query_prot_names[prot_i])
                query_prot_records_filtr.append(query_prot_records[prot_i])
            else:
                repeated_hogid_num += 1 
                # for development
                #print("repeated hogid",query_hogids[prot_i], " for protein ",query_prot_names[prot_i])
                # now we keep the first protein query when these are repeated


        query_hogids_filtr_species.append(query_hogids_filtr)
        query_prot_names_filtr_species.append(query_prot_names_filtr)
        query_prot_records_filtr_species.append(query_prot_records_filtr)


        num_query_filtr = len(query_hogids_filtr)
        #print("Number of prot queries after filtering is",num_query_filtr,"\n")

    

    return (query_hogids_filtr_species, query_prot_names_filtr_species, query_prot_records_filtr_species )
    
    

In [ ]:

def gather_OG(query_species_names, query_hogids_filtr_species, query_prot_names_filtr_species, query_prot_records_filtr_species):

    ############ Extracting the most frequent OG  ########
    #####################################################

    #dict (oma_group_nr -> dict(species, [proteins]))
    #Og[555] = {homo_erectus: [blabla, blublu], yellow_bird: [P52134], brown_bear: [P2121,B53223]}

    OGs_queries = {}

    # hog_OG_map = {}

    mostFrequent_OG_list_species = []

    frq_most_frequent_og_list_all = [] # for development
    
    query_species_num = len(query_species_names)  
    for species_i in  range(query_species_num):

        query_species_name = query_species_names[species_i]
        #print("\n",query_species_name)

        query_hogids_filtr = query_hogids_filtr_species[species_i]
        query_prot_names_filtr = query_prot_names_filtr_species[species_i]
        query_prot_records_filtr = query_prot_records_filtr_species[species_i]

        mostFrequent_OG_list=[]

        num_query_filtr = len(query_hogids_filtr)
        for  item_idx in range(num_query_filtr): #

            #query_protein = query_prot_names_filtr[item_idx]
            seqRecords_query =  query_prot_records_filtr[item_idx]
            seqRecords_query_edited = SeqRecord(Seq(str(seqRecords_query.seq)), query_species_name, '', '')
            #print(seqRecords_query_edited)

            hog_id= query_hogids_filtr[item_idx]

            if not hog_id in hog_OG_map:   # Caculitng  most frq OG for the new hog
                mostFrequent_OG= -1
                hog_OG_map[hog_id]=mostFrequent_OG

            else:  # hog_id is in hog_OG_map dic
                #print("using the hog-og-map")
                mostFrequent_OG = hog_OG_map[hog_id]

            if mostFrequent_OG in OGs_queries:
                OGs_queries_k = OGs_queries[mostFrequent_OG]

                if not query_species_name in OGs_queries_k:
                    OGs_queries_k[query_species_name] = seqRecords_query_edited
                    OGs_queries[mostFrequent_OG] = OGs_queries_k
            else:
                OGs_queries[mostFrequent_OG] = {query_species_name: seqRecords_query_edited} # query_protein = query_prot_names_filtr[item_idx]
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Needed HOH-OG map is extracted from the map file.") 
    
    return OGs_queries
    

In [ ]:

def combine_OG_query(OGs_queries, oma_db,threshold_least_query_sepecies_in_OG):
    
    ########## Combine proteins of OG with queries ##################
    #################################################################
    
    seqRecords_OG_queries = []
    seqRecords_all = []
    for OG_q in OGs_queries.keys():  # OG found in the query

        dic_species_prot = OGs_queries[OG_q]
        if len(dic_species_prot) >threshold_least_query_sepecies_in_OG:

            seqRecords_query_edited_all = []
            for query_species_name,seqRecords_query_edited  in dic_species_prot.items():
                #print(seqRecords_query_edited)
                seqRecords_query_edited_all.append(seqRecords_query_edited) 


            mostFrequent_OG = OG_q
            if mostFrequent_OG != -1:
                OG_members = oma_db.oma_group_members(mostFrequent_OG)
                proteins_object_OG = [db.ProteinEntry(oma_db, pr) for pr in OG_members]  # the protein IDs of og members
                 # covnert to biopython objects
                seqRecords_OG=[SeqRecord(Seq(pr.sequence),str(pr.genome.uniprot_species_code),'','') for pr in proteins_object_OG]

                seqRecords_OG_queries =seqRecords_OG + seqRecords_query_edited_all
                current_time = datetime.now().strftime("%H:%M:%S")
                #print("length of OG",mostFrequent_OG,"was",len(seqRecords_OG),",now is",len(seqRecords_OG_queries))
                print(current_time, " - Combining an OG with length of ",len(seqRecords_OG),"\t with a query is just finished.")

            seqRecords_all.append(seqRecords_OG_queries)


    current_time = datetime.now().strftime("%H:%M:%S")
    print("\n", current_time, "- Combining queries with OG is finished! number of OGs",len(seqRecords_all)) # 
    return(seqRecords_all)


In [ ]:

def run_msa_OG(seqRecords_OG_queries):
    ############## MSA  ##############
    ##################################
    #current_time = datetime.now().strftime("%H:%M:%S")
    #print(current_time, "- working on new OG with length of ",len(seqRecords_OG_queries))

    wrapper_mafft = mafft.Mafft(seqRecords_OG_queries,datatype="PROTEIN") 
    # MAfft error: Alphabet 'U' is unknown. -> add --anysymbol argument needed to define in the sourse code
    # workaround sed "s/U/X/g"
    
    wrapper_mafft.options.options['--retree'].set_value(1)


    run_mafft = wrapper_mafft() # it's wrapper  storing the result  and time 
    time_taken_mafft = wrapper_mafft.elapsed_time

    result_mafft = wrapper_mafft.result 
    time_taken_mafft2 = wrapper_mafft.elapsed_time
    
    current_time = datetime.now().strftime("%H:%M:%S")
    #print(current_time,"- time elapsed for MSA: ",time_taken_mafft2)
    print(current_time,"- MSA for an OG is just finished: ",time_taken_mafft2)

    return(result_mafft)
   


In [ ]:

def concatante_alignments(result_mafft_all_species, project_folder):
    ############## Concatante alignments  ##############
    ####################################################

    #alignments= result_maf2_all

    alignments= result_mafft_all_species
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- alignments len",len(alignments))
    #print([len(aln) for aln in alignments ])
    #print([len(seq) for aln in alignments for seq in aln])

    all_labels_raw = [seq.id for aln in alignments for seq in aln]
    all_labels = set(all_labels_raw)
    print("ids: ",len(all_labels),len(all_labels_raw))

    # Make a dictionary to store info as we go along
    # (defaultdict is convenient -- asking for a missing key gives back an empty list)
    concat_buf = defaultdict(list)

    # Assume all alignments have same alphabet
    alphabet = alignments[0]._alphabet

    for aln in alignments:
        length = aln.get_alignment_length()
        #print("length",length)
        # check if any labels are missing in the current alignment
        these_labels = set(rec.id for rec in aln)
        missing = all_labels - these_labels
        #print(missing)
        # if any are missing, create unknown data of the right length,
        # stuff the string representation into the concat_buf dict
        for label in missing:
            new_seq = UnknownSeq(length, alphabet=alphabet)
            concat_buf[label].append(str(new_seq))

        # else stuff the string representation into the concat_buf dict
        for rec in aln:
            concat_buf[rec.id].append(str(rec.seq))

    # Stitch all the substrings together using join (most efficient way),
    # and build the Biopython data structures Seq, SeqRecord and MultipleSeqAlignment
    msa = MultipleSeqAlignment(SeqRecord(Seq(''.join(seq_arr), alphabet=alphabet), id=label)
                                for (label, seq_arr) in concat_buf.items())



    out_name_msa=project_folder+"_msa_concatanated.txt"
    handle_msa_fasta = open(out_name_msa,"w")
    SeqIO.write(msa, handle_msa_fasta,"fasta")
    handle_msa_fasta.close()
    
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- ", len(msa),msa.get_alignment_length()) # super matrix size
    
    return msa
    
    

In [ ]:


def msa_filter_row(msa,project_folder,tresh_ratio_gap_row,query_species_names):

    
    msa_filtered_row = [] # msa_fltr
    ratio_records=[]
    for record in msa:
        species_name = record.id
        seq = record.seq
        seqLen = len(record)
        
        gap_count=seq.count("-") + seq.count("?") + seq.count(".") +seq.count("~")
                
        ratio_record_nongap= 1-gap_count/seqLen
        ratio_records.append(round(ratio_record_nongap,3))

        if ratio_record_nongap > tresh_ratio_gap_row:
            msa_filtered_row.append(record)
        elif species_name in query_species_names : 
            msa_filtered_row.append(record)
            current_time = datetime.now().strftime("%H:%M:%S")
            print(current_time, "- Many row-wise gap for query",species_name,"with a ratio of",ratio_record_nongap) 

    current_time = datetime.now().strftime("%H:%M:%S")

    print(current_time, "- Row-wise filtering of MSA is finished.") 
    print(current_time, "- Out of ",len(msa),"species,",len(msa_filtered_row),"species (row of msa) remained.")


    out_name_msa=project_folder+"_msa_concatanated_filtered_row_"+str(tresh_ratio_gap_row)+".txt"
    handle_msa_fasta = open(out_name_msa,"w")
    SeqIO.write(msa_filtered_row, handle_msa_fasta,"fasta")
    handle_msa_fasta.close()
    
    print(current_time, "- MSA Row-wise filtered stored in file.") # super matrix size
    
    
    return msa_filtered_row
    

In [ ]:

def msa_filter_col(msa_filtered_row, tresh_ratio_gap_col):

    ratio_col_all = []

    length_record= len(msa_filtered_row[1])
    num_records = len(msa_filtered_row)


    keep_cols = []
    for col_i in range(length_record):  # inspired by https://github.com/andreas-wilm/compbio-utils/blob/master/prune_aln_cols.py 

        col_values = [record.seq[col_i] for record in msa_filtered_row]

        gap_count=col_values.count("-") + col_values.count("?") + col_values.count(".") +col_values.count("~")

        ratio_col_nongap = 1- gap_count/num_records
        ratio_col_all.append(ratio_col_nongap)
        if ratio_col_nongap  > tresh_ratio_gap_col:
            keep_cols.append(col_i)


    plt.hist(ratio_col_all,bins=100) # , bins=10
    #plt.show()
    plt.savefig("./__ratio_col.pdf")

    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Columns indecis extracted. Out of ", length_record,"columns,",len(keep_cols),"is remained.") 

    msa_filtered_row_col = []

    for record in msa_filtered_row :
        record_seq = str(record.seq)

        record_seq_edited  = ''.join([record_seq[i] for i in keep_cols  ])
        record_edited= SeqRecord(Seq(record_seq_edited), record.id, '', '')
        msa_filtered_row_col.append(record_edited)                         


    
    out_name_msa=project_folder+"_msa_concatanated_filtered_row_col_"+str(tresh_ratio_gap_col)+".txt"
    handle_msa_fasta = open(out_name_msa,"w")
    SeqIO.write(msa_filtered_row_col, handle_msa_fasta,"fasta")
    handle_msa_fasta.close()
    
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Column-wise filtering of MSA is finished",len(msa_filtered_row_col),len(msa_filtered_row_col[0])) 
       
    #msa_filtered_row_col = MultipleSeqAlignment(msa_filtered_row_col)
    return msa_filtered_row_col



In [ ]:

def draw_tree(msa, project_folder):
    ############## Tree inference  ###################
    ##################################################

    wrapper_tree=fasttree.Fasttree(msa,datatype="PROTEIN")
    wrapper_tree.options.options['-fastest']    
    result_tree1 = wrapper_tree()

    time_taken_tree = wrapper_tree.elapsed_time 
    time_taken_tree

    result_tree2 = wrapper_tree.result
    tree_nwk=str(result_tree2["tree"])
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- ",len(tree_nwk))

    out_name_tree=project_folder+"_tree.txt"
    file1 = open(out_name_tree,"w")
    file1.write(tree_nwk)
    file1.close() 
    return tree_nwk

In [ ]:



if __name__ == "__main__":
    (oma_db, hog_OG_map, list_speices) = parse_oma(oma_database_address, hog_og_map_address)


    (query_species_names, query_prot_records_species) = parse_proteome(project_folder, list_speices)
    (query_prot_names_species, query_hogids_species) = parse_hogmap_omamer(project_folder,query_species_names )


    (query_hogids_filtr_species, query_prot_names_filtr_species, query_prot_records_filtr_species) = extract_unique_hog(query_species_names,query_hogids_species, query_prot_names_species,query_prot_records_species)

    OGs_queries = gather_OG(query_species_names, query_hogids_filtr_species, query_prot_names_filtr_species, query_prot_records_filtr_species)

    #seqRecords_all = combine_OG_query(OGs_queries, oma_db)
    
    threshold_least_query_sepecies_in_OG = 15
    seqRecords_all = combine_OG_query(OGs_queries, oma_db,threshold_least_query_sepecies_in_OG)
    #combine_OG_query(OGs_queries, oma_db)
    
    
    num_OGs= len(seqRecords_all)
    
    

In [ ]:
    
    iterotr_OGs = 0 
    
    result_mafft_all_species=[]
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Parallel msa is started for ",len(seqRecords_all)," OGs.")
    
    number_max_workers = 1
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=number_max_workers) as executor: 
        for seqRecords_OG_queries, output_values in zip(seqRecords_all, executor.map(run_msa_OG, seqRecords_all)):
            result_mafft_all_species.append(output_values)

    msa= concatante_alignments(result_mafft_all_species, project_folder)
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- all msa are concatanated")
    
    


In [ ]:
#     current_time = datetime.now().strftime("%H:%M:%S")
#     print(current_time, "- Tree inference is started")

#     concurrent.futures.as_completed(executor)
#     tree_nwk = draw_tree(msa, project_folder)
    
    
#     print(current_time, "- Tree inference is finsiehd. Thanks for your patience!")
#     tree_nwk

In [ ]:
    #msa_origin =list(msa)[:]

In [ ]:
#     #add="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/_msa_concatanated.txt"
#     add="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/_msa_concatanated.txt"
#     msa = AlignIO.read(add,"fasta")
#     print(len(msa),len(msa[0]))


In [ ]:
##    Instead of running MSA and using concatatntion. you could used pre_calculated concat msa

#     project_folder =  "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/" # _msa_concatanated_hogmapX.txt"
#     project_files = listdir(project_folder)
#     query_species_names = []
#     for file in project_files:
#         if file.split(".")[-1]=="fa":
#             file_name_split = file.split(".")[:-1]
#             query_species_names.append('.'.join(file_name_split))


#     msa_input = project_folder+"_msa_concatanated.txt"
#     msa = AlignIO.read(msa_input,"fasta")
#     print("finish reading file",len(msa),len(msa[0]))


In [ ]:
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Row-wise filtering of MSA is started.") 
       
    tresh_ratio_gap_row = 0.02
    msa_filtered_row = msa_filter_row(msa,project_folder,tresh_ratio_gap_row,query_species_names)

    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- Column-wise filtering of MSA is started.") 
    
    tresh_ratio_gap_col = 0.1
    msa_filtered_row_col=  msa_filter_col(msa_filtered_row, tresh_ratio_gap_col)
    
    


In [ ]:
import ete3

In [ ]:
    msa_input = project_folder+"_msa_concatanated.txt"
    msa = AlignIO.read(msa_input,"fasta")
    print("finish reading file",len(msa),len(msa[0]))


In [ ]:
msa2=msa[:3]

In [ ]:
a=draw_tree(msa2,project_folder)

In [25]:


def read_taxonID_map(omaID_address,bird6ID_address):
    
    omaID_file = open(omaID_address,'r')
    taxonID_omaID={}
    omaID_taxonID={}
    for line in omaID_file:
        line_strip = line.strip()
        if line_strip.startswith('#'):
            pass
            #header_lines_list.append(line_strip)
        else:
            line_parts=line_strip.split('\t')

            omaID = line_parts[0]
            taxonID = int(line_parts[1])
            taxonID_omaID[taxonID]=omaID
            omaID_taxonID[omaID]=taxonID
            
    omaID_file.close()
        
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- The map for taxonID omaID of",len(taxonID_omaID),"records have read.") 
    
    
    bird6ID_file = open(bird6ID_address,'r')
    taxonID_bird6ID={}
    bird6ID_taxonID= {}
    for line in bird6ID_file:
        line_strip = line.strip()
        if line_strip.startswith('Or'):
            pass
            line_strip1=line_strip
            #header_lines_list.append(line_strip)
        else:
            line_parts=line_strip.split('\t')

            bird6ID = line_parts[6]
            taxonID = int(line_parts[8])
            taxonID_bird6ID[taxonID]=bird6ID
            bird6ID_taxonID[bird6ID]=taxonID
            
    bird6ID_file.close()
        
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- The map for taxonID bird6ID of",len(taxonID_bird6ID),"records have read.") 
  
    return (taxonID_omaID,taxonID_bird6ID,omaID_taxonID,bird6ID_taxonID)
    
            

In [26]:
len(taxonID_list)

2778

In [27]:
(taxonID_omaID,taxonID_bird6ID,omaID_taxonID,bird6ID_taxonID) = read_taxonID_map(omaID_address,bird6ID_address)
taxonID_map = {**taxonID_omaID,**taxonID_bird6ID }







#print(len(taxonID_omaID),len(taxonID_bird6ID),len(taxonID_map))
taxonID_list= list(taxonID_map.keys())

import ete3
ncbi = ete3.NCBITaxa()  # first time download in ~/.etetoolkit/
ncbi_sub_tree = ncbi.get_topology(taxonID_list)

for node in ncbi_sub_tree.traverse(strategy="postorder"):
    node.name2 = node.name
    if node.is_leaf() and int(node.name) in taxonID_map: # why ?
        node.name = taxonID_map[int(node.name)]

current_time = datetime.now().strftime("%H:%M:%S")
print(current_time, "- The NCBI taxanomy is read and the leaves names changed to OMA/bird6 ID containing",len(ncbi_sub_tree),".") 
#print(ncbi_sub_tree.get_ascii(attributes=["name"]))     
#ncbi_sub_tree.write() 

11:09:50 - The map for taxonID omaID of 2424 records have read.
11:09:50 - The map for taxonID bird6ID of 363 records have read.
11:10:05 - The NCBI taxanomy is read and the leaves names changed to OMA/bird6 ID containing 2756 .


In [8]:
ncbi_sub_tree.write()

'((((((((DESCC:1,DESRM:1,DESRL:1)1:1,((DESHD:1,DESHY:1)1:1,DESDL:1)1:1,(DESMD:1,DESOD:1,DESAJ:1)1:1,PELTS:1,DESAS:1,DESAP:1,THEPJ:1,DESK7:1,SYNGF:1)1:1,((RUMC1:1,RUMA7:1)1:1,((HUNT2:1,HUNT1:1)1:1,HUNCD:1)1:1,RUMCH:1,MAGIU:1,OSCVS:1,ETHHY:1)1:1,((((CLOBB:1,CLOB6:1,CLOBK:1)1:1,(CLOBJ:1,CLOBM:1,CLOB1:1,CLOBH:1)1:1,(CLOB2:1,CLOBL:1)1:1,CLOB0:1,CLOBA:1)1:1,(CLOS4:1,CLOSS:1)1:1,(CLOAB:1,CLOAE:1)1:1,(CLOPS:1,CLOPE:1,CLOP1:1)1:1,CLONN:1,CLOLD:1,CLOB8:1,CLOK5:1,CLOC7:1,CLOTE:1)1:1,(ALKOO:1,ALKMQ:1)1:1,ARTSS:1)1:1,((CLOD6:1,CLODR:1)1:1,FILAD:1,ACESD:1)1:1,(ANAHA:1,CELLD:1,ROSHA:1,CLOSW:1,LACP7:1,AGARV:1,EUBE2:1,BUTPB:1)1:1,(EUBLK:1,ACEWD:1)1:1,(SYNLT:1,SYNWW:1)1:1,((SULAD:1,SULAT:1)1:1,THEM7:1)1:1,SYMTH:1,HELMI:1)1:1,((((THEPX:1,THESX:1)1:1,THEP3:1,THEIA:1,THEM3:1)1:1,THEPS:1,CARHZ:1,MOOTA:1,CALS4:1)1:1,((CALH1:1,CALKI:1,CALK2:1,CALOW:1,CALOO:1,CALBD:1,CALS8:1)1:1,(THESW:1,THEXL:1,THETC:1)1:1)1:1,MAHA5:1)1:1,((ACEAZ:1,HALHC:1)1:1,((HALHG:1,HALPG:1)1:1,HALOH:1)1:1)1:1,(TEPAE:1,THEOJ:1)1:1,NATTJ:1

In [42]:
bird_hog_tree_address="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/hogmapX/out4/_tree_filt_7.txt"
#"/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/_tree__msa_concatanated_filtered_row_col_0.00101.txt"
bird_hog_tree= ete3.Tree(bird_hog_tree_address)





In [11]:
a_count=0
for node in ncbi_sub_tree.traverse(strategy="postorder"):
    node.name2 = node.name
    if node.is_leaf() : # why ?
        a_count+=1


In [12]:
a_count

2756

In [13]:
b_count=0
for node in bird_hog_tree.traverse(strategy="postorder"):
    if node.is_leaf() : # why ?
        b_count+=1
b_count

411

In [9]:
bird_hog_tree.write()

'(((RHIDAH:0.00916118,CHAPAP:0.00759941)1:0.00218398,(DICMEG:0.00764643,(MYIHEB:0.00930175,((LANLUD:0.0100277,(APHCOE:0.00418719,(CORMON:0.00242676,(CORCOR:0.00159595,CORBRA:0.00301169)1:0.000693276)1:0.00236036)1:0.00371554)1:0.000713619,(STRCIN:0.00728446,(PARRAG:0.00655149,IFRKOW:0.00728625)1:5.6528e-05)1:6.8882e-05)1:6.8104e-05)0:8.1981e-05)1:7.9969e-05)1:0.00134414,(EDOCOE:0.00927021,DAPCHR:0.012291)1:0.000149208,((MOHOCH:0.00887504,(PTILEU:0.0117964,(((CALWIL:0.0104743,NOTCIN:0.0163835)1:0.000624526,(CNELOR:0.0106946,(MELVER:0.0112781,((((ANTMIN:0.0143893,(POEATR:0.00417223,(PSEHUM:0.00339711,(PARMJ:0.00208132,PARMAJ:0.00069855)1:0.00289579)1:0.000804694)1:0.00734026)1:0.00172581,((ALACHE:0.0166442,PANBIA:0.0165384)1:0.00104961,(NICCHL:0.0122264,(SYLVIR:0.0139867,((HIRRUS:0.0127127,(((BRAATR:0.00700473,PYCJOC:0.00676696)1:0.00539269,((SINWEB:0.00855782,(SYLATR:0.00541378,SYLBOR:0.00456145)1:0.00584524)1:0.000586819,((STEDEN:0.00419501,(ZOSHYP:0.0023303,ZOSLAT:0.00328568)1:0.00242

In [7]:
out = bird_hog_tree.robinson_foulds(ncbi_sub_tree,unrooted_trees=True)  # , expand_polytomies = True
(rf, rf_max, common_attrs, names, edges_t1, edges_t2, discarded_edges_t1) =out
print("RF distance is %s over a total of %s" %(rf, rf_max))


#print "Partitions in tree2 that were not found in tree1:", parts_t1 - parts_t2
#print "Partitions in tree1 that were not found in tree2:", parts_t2 - parts_t1

RF distance is 345 over a total of 567


In [132]:
for node in bird_hog_tree.traverse(strategy="postorder"):
    node.name2 = node.name
    if node.is_leaf(): 
        if "2000_X" in node.name:
            node.name = node.name[:6]
            #print(node.name)

    
(rf, rf_max, common_attrs, names, edges_t1, edges_t2, discarded_edges_t1) =out
print("RF distance is %s over a total of %s" %(rf, rf_max))


#print "Partitions in tree2 that were not found in tree1:", parts_t1 - parts_t2
#print "Partitions in tree1 that were not found in tree2:", parts_t2 - parts_t1

RF distance is 1383 over a total of 2585


In [10]:
list_leaf= []
for node in bird_hog_tree.traverse(strategy="postorder"):
    if node.is_leaf(): 
        list_leaf.append(node.name)


In [138]:
#ncbi_sub_tree.write() 

In [140]:

node_name_list= []
for node in bird_hog_tree.traverse(strategy="postorder"):
    node.name2 = node.name
    if node.is_leaf(): 
        if "2000_X" in node.name:
            node.name = node.name[:6]
        node_name_list.append(node.name)
            
            
            

In [164]:
not_list=['PHYPR', 'GUITH', 'CYAME', 'PUCGR', 'COPCI', 'SCHCO', 'PHACH', 'EMEND', 'PHAND', 'NECHA', 'VERDA', 'PROLT', 'BATDE', 'CRYPV', 'TOXGO', 'MYCCU', 'BURSC', 'METAA', 'AGRSH', 'MELGA', 'CHICK', 'ANAPL', 'MELUD', 'FICAL', 'PARMJ', 'TAEGU', 'JUNHY', 'STAA3', 'LACRD', 'SERCA', 'ANAPLA']

#bird6ID_list=list(taxonID_bird6ID.keys()) taxon
node_name_list_sub = [i for i in node_name_list if i not in not_list and i in bird6ID_list]

In [165]:
node_name_list_sub

['APTFOR',
 'ALELAT',
 'APTROW',
 'APTOWE',
 'ALCTOR',
 'ASASCU',
 'AMAGUI',
 'APHCOE',
 'AGEPHO',
 'AEGBEN',
 'APAVIT',
 'ACACHL',
 'ANSCYG',
 'ANAZON',
 'APTHAA',
 'AGAROS',
 'AQUCHR',
 'ALOBEC',
 'ATLROG',
 'APTAUS',
 'ANTMIN',
 'AEGCAU',
 'ANTCAR',
 'ATRCLA',
 'ALERUF',
 'ALACHE',
 'ACRARU',
 'ANSSEM',
 'ARDKOR',
 'ARAGUA',
 'AREINT',
 'ANHRUF',
 'ANHANH']

In [166]:

ncbi_sub_tree_original= ncbi_sub_tree
ncbi_sub_tree.prune(node_name_list_sub)


In [9]:
ncbi_sub_tree.write()

'((((((((DESCC:1,DESRM:1,DESRL:1)1:1,((DESHD:1,DESHY:1)1:1,DESDL:1)1:1,(DESMD:1,DESOD:1,DESAJ:1)1:1,PELTS:1,DESAS:1,DESAP:1,THEPJ:1,DESK7:1,SYNGF:1)1:1,((RUMC1:1,RUMA7:1)1:1,((HUNT2:1,HUNT1:1)1:1,HUNCD:1)1:1,RUMCH:1,MAGIU:1,OSCVS:1,ETHHY:1)1:1,((((CLOBB:1,CLOB6:1,CLOBK:1)1:1,(CLOBJ:1,CLOBM:1,CLOB1:1,CLOBH:1)1:1,(CLOB2:1,CLOBL:1)1:1,CLOB0:1,CLOBA:1)1:1,(CLOS4:1,CLOSS:1)1:1,(CLOAB:1,CLOAE:1)1:1,(CLOPS:1,CLOPE:1,CLOP1:1)1:1,CLONN:1,CLOLD:1,CLOB8:1,CLOK5:1,CLOC7:1,CLOTE:1)1:1,(ALKOO:1,ALKMQ:1)1:1,ARTSS:1)1:1,((CLOD6:1,CLODR:1)1:1,FILAD:1,ACESD:1)1:1,(ANAHA:1,CELLD:1,ROSHA:1,CLOSW:1,LACP7:1,AGARV:1,EUBE2:1,BUTPB:1)1:1,(EUBLK:1,ACEWD:1)1:1,(SYNLT:1,SYNWW:1)1:1,((SULAD:1,SULAT:1)1:1,THEM7:1)1:1,SYMTH:1,HELMI:1)1:1,((((THEPX:1,THESX:1)1:1,THEP3:1,THEIA:1,THEM3:1)1:1,THEPS:1,CARHZ:1,MOOTA:1,CALS4:1)1:1,((CALH1:1,CALKI:1,CALK2:1,CALOW:1,CALOO:1,CALBD:1,CALS8:1)1:1,(THESW:1,THEXL:1,THETC:1)1:1)1:1,MAHA5:1)1:1,((ACEAZ:1,HALHC:1)1:1,((HALHG:1,HALPG:1)1:1,HALOH:1)1:1)1:1,(TEPAE:1,THEOJ:1)1:1,NATTJ:1

In [163]:
bird6ID_list=[]
bird6ID_file = open(bird6ID_address,'r')
taxonID_bird6ID={}
for line in bird6ID_file:
    line_strip = line.strip()
    if line_strip.startswith('Or'):
        pass
        line_strip1=line_strip
        #header_lines_list.append(line_strip)
    else:
        line_parts=line_strip.split('\t')

        bird6ID = line_parts[6]
        bird6ID_list.append(bird6ID)


In [162]:
line_parts[6]

['Passeriformes',
 'Zosteropidae B',
 'Zosterops',
 'Zosterops lateralis',
 'Silver-eye',
 'NCBI-003',
 'ZOSLAT',
 '15251',
 '43581']

In [ ]:
t = Tree('((((H,K),(F,I)G),E),((L,(N,Q)O),(P,S)));', format=1)
print("Original tree looks like this:")
print(t)

t.prune(["H","F","E","Q", "P"])
print("Pruned tree"
print t

In [168]:

bird_hog_tree_original= bird_hog_tree
bird_hog_tree.prune(node_name_list_sub)


In [169]:
bird_hog_tree.write()

'(APTFOR:1.45395,(((ALELAT:0.0938539,((APTROW:0.0212476,APTOWE:0.0431023)1:0.0691344,ALCTOR:0.073543)0.972:0.0417979)1:0.126189,ASASCU:0.0902211)0.998:0.025349,(((APHCOE:0.452866,AGEPHO:0.172617)1:0.0605827,AMAGUI:0.0481192)0.231:0.00638122,((AEGBEN:0.228914,((APAVIT:0.259342,(ACACHL:0.172978,(ANSCYG:0.054373,ANAZON:0.0206031)0.232:0.0451732)0.261:0.110195)0.796:0.041677,(APTHAA:0.692812,AGAROS:0.41341)0.999:0.670898)0.975:0.0818402)0.993:0.0852679,(((AQUCHR:0.258063,ALOBEC:0.613767)0.974:0.155622,(ATLROG:0.498934,APTAUS:0.366763)0.828:0.0314773)0.993:0.11265,((ANTMIN:0.764073,((AEGCAU:0.0573052,ANTCAR:0.64669)0.984:0.0735707,(ATRCLA:0.122335,((ALERUF:0.118689,ALACHE:0.111168)0.374:0.0793286,(ACRARU:0.026524,ANSSEM:0.109276)0.978:0.0883819)1:0.14146)0.564:0.0845237)1:0.14491)0.908:0.0463586,(ARDKOR:0.642256,((ARAGUA:0.110572,AREINT:0.123587)0.991:0.123277,(ANHRUF:0.224563,ANHANH:0.473539)0.98:0.151921)0.838:0.0373177)0.936:0.0474706)0.999:0.0768067)0.988:0.0692005)1:0.239339)0.996:0.02

In [170]:
out = bird_hog_tree.robinson_foulds(ncbi_sub_tree,unrooted_trees=True)  # , expand_polytomies = True
(rf, rf_max, common_attrs, names, edges_t1, edges_t2, discarded_edges_t1) =out
print("RF distance is %s over a total of %s" %(rf, rf_max))


#print "Partitions in tree2 that were not found in tree1:", parts_t1 - parts_t2
#print "Partitions in tree1 that were not found in tree2:", parts_t2 - parts_t1

RF distance is 41 over a total of 43


In [18]:

taxonID_from_oma_list_dic={}

list_leaf= []
for node in bird_hog_tree.traverse(strategy="postorder"):
    if node.is_leaf(): 
        list_leaf.append(node.name)

taxonID_from_oma_list =[]
omaID_file = open(omaID_address,'r')
# taxonID_omaID_small ={}
for line in omaID_file:
    line_strip = line.strip()
    if line_strip.startswith('#'):
        pass
        #header_lines_list.append(line_strip)
    else:
        line_parts=line_strip.split('\t')         
        omaID = line_parts[0]
        taxonID = int(line_parts[1])
        
        if omaID in list_leaf:
#             taxonID_omaID_small[taxonID]=omaID
            taxonID_from_oma_list.append(taxonID)

omaID_file.close()


taxonID_from_bird_list =[]
bird6ID_file = open(bird6ID_address,'r')
# taxonID_bird6ID={}
for line in bird6ID_file:
    line_strip = line.strip()
    if line_strip.startswith('Or'):
        pass
        line_strip1=line_strip
        #header_lines_list.append(line_strip)
    else:
        line_parts=line_strip.split('\t')

        bird6ID = line_parts[6]        
        taxonID = int(line_parts[8])
        if bird6ID in list_leaf:
            taxonID_from_bird_list.append(taxonID)
#         taxonID_bird6ID[taxonID]=bird6ID
        
        
        
bird6ID_file.close()

# current_time = datetime.now().strftime("%H:%M:%S")
# print(current_time, "- The map for taxonID bird6ID of",len(taxonID_bird6ID),"records have read.") 

taxonID_list_all =  taxonID_from_bird_list + taxonID_from_oma_list

print(len(taxonID_list_all))

ncbi_sub_tree_small = ncbi.get_topology(taxonID_list_all)

for node in ncbi_sub_tree_small.traverse(strategy="postorder"):
    node.name2 = node.name
    if node.is_leaf() and int(node.name) in taxonID_map: # why ?
        node.name = taxonID_map[int(node.name)]

        
        
ncbi_sub_tree_small.write()

411


'((((((BUCCAP:1,GALDEA:1)1:1,((((TAEGUT:1,LONSTR:1)1:1,(VIDCHA:1,VIDMAC:1)1:1)1:1,(((LOXLEU:1,LOXCUR:1)1:1,SERCAN:1)1:1,((CALORN:1,EMBFUC:1)1:1,(CARCAR:1,PASAMO:1,PHEMEL:1)1:1)1:1,UROPYL:1,CHLVIR:1,PEUTAE:1,HEMWIL:1)1:1,((PRUFUL:1,PRUHIM:1)1:1,PASDOM:1)1:1,(PROCAF:1,LEPASP:1)1:1,(AGEPHO:1,QUIMEX:1,MOLATE:1)1:1,(SETCOR:1,SETKIR:1)1:1,OREARF:1,MELVER:1,MOTALB:1,DICEXI:1,PLONIG:1)1:1,(((ZOSLAT:1,ZOSHYP:1)1:1,HYPCIN:1,STEDEN:1)1:1,(PORRUF:1,ILLCLE:1,PTEMEL:1,ERPZAN:1,POSRUF:1,MYSCRO:1,OXYMAD:1)1:1,(BRAATR:1,PYCJOC:1,NICCHL:1)1:1,((ACRARU:1,CETCET:1,HIPICT:1,SYLVIR:1,HYLPRA:1)1:1,(PANBIA:1,REGSAT:1)1:1,(SYLATR:1,SYLBOR:1)1:1,SINWEB:1)1:1,(PHYTRO:1,RHASIB:1)1:1,HIRRUS:1,ALACHE:1,LOCOCH:1,HORVUL:1,CISJUN:1,LEILUT:1,AEGCAU:1)1:1,(((CHLCYA:1,CHLHAR:1)1:1,IRECYA:1)1:1,((CORMON:1,CORCOR:1,CORBRA:1)1:1,CNELOR:1,APHCOE:1,ORIORI:1)1:1,(ALERUF:1,EULNIG:1,RHALEU:1,DAPCHR:1,FALFRO:1,PACPHI:1)1:1,(IFRKOW:1,PTILEU:1)1:1,(ERYMCC:1,DICMEG:1,MACNIG:1)1:1,LANLUD:1,MYIHEB:1,CALWIL:1,DRYGAM:1,EDOCOE:1,PARRAG:1

In [19]:
len(ncbi_sub_tree_small),len(bird_hog_tree)

(405, 411)

In [24]:
out = ncbi_sub_tree_small.robinson_foulds(bird_hog_tree,unrooted_trees=True)  # , expand_polytomies = True
(rf, rf_max, common_attrs, names, edges_t1, edges_t2, discarded_edges_t1) =out
print("RF distance is %s over a total of %s" %(rf, rf_max))



RF distance is 345 over a total of 569


In [20]:
len(taxonID_from_bird_list),len(taxonID_list_all),len()

(363, 411, 405)

In [31]:
taxonID_list_all =  taxonID_from_bird_list + taxonID_from_oma_list
#taxonID_list_all


In [21]:
taxonID_list_al_unq= set(taxonID_list_all)
for i in taxonID_list_al_unq:
    if taxonID_list_all.count(i)>1:
        print(i)
    

59729
59894
8839
9031
9135
9157


In [ ]:
ncbi_sub_tree_original= ncbi_sub_tree
ncbi_sub_tree.prune(node_name_list_sub)

In [29]:
for i in [59729,59894,8839,9031,9135,9157]:
    if i in taxonID_omaID:
        print(taxonID_omaID[i])
    if i in taxonID_bird6ID:
        print(taxonID_bird6ID[i])


TAEGU
TAEGUT
FICAL
FICALB
ANAPL
ANAPLA
CHICK
GALGAL
SERCA
SERCAN
PARMJ
PARMAJ


In [ ]:
ncbi_sub_tree_original= ncbi_sub_tree
ncbi_sub_tree.prune(node_name_list_sub)

In [ ]:
            taxonID_bird6ID[taxonID]=bird6ID
            bird6ID_taxonID[bird6ID]=taxonID
        
        

In [54]:
five_repetead=set(["TAEGU","FICAL","ANAPL","CHICK","SERCA","PARMJ"])

node_uniq=list(set(list(bird6ID_taxonID.keys())+list(omaID_taxonID.keys()))-five_repetead)
len(node_uniq)

2781

In [ ]:
taxonID_map = {**taxonID_omaID,**taxonID_bird6ID }

In [48]:


bird_hog_tree.nodes()


AttributeError: 'TreeNode' object has no attribute 'nodes'

In [55]:
    #bird_hog_tree=

ValueError: Node names not found: ['PSEUT', 'LISIN', 'GEOBB', 'PETMO', 'LACRJ', 'TRYB2', 'HELP4', 'LACLN', 'POLNS', 'PSEUX', 'NAUPA', 'NEIMW', 'ORYRU', 'LACPS', 'ANAMM', 'HELPG', 'OENOB', 'AZOC5', 'PENRW', 'ENTBF', 'CORCM', 'PELHB', 'BLASD', 'RHOS5', 'EHRCJ', 'DRORH', 'BORRA', 'MYCT3', 'CLONN', 'MYCAS', 'SALBC', 'RHOSU', 'ABSGL', 'THEP3', 'BUCA0', 'TETBL', 'CALK2', 'AZOOP', 'BURP1', 'MYCA1', 'METRJ', 'CHRVO', 'STRE4', 'SALSV', 'NOCDD', 'LACCD', 'METAX', 'STRT2', 'LACRL', 'STRSY', 'METM5', 'CIOSA', 'PECCP', 'CRYPV', 'AYWBP', 'SYNR3', 'ZOSMR', 'SPHS2', 'BRELN', 'LEUCJ', 'MICSL', 'PARPJ', 'METH6', 'LACSS', 'BORPC', 'ECO7I', 'AEQSU', 'AURPU', 'RHOP5', 'SALPS', 'ALTAL', 'THASP', 'CALNY', 'HELPQ', 'DRONM', 'PROMH', 'PAESJ', 'HALS3', 'TROW8', 'LOTJA', 'BIFAA', 'SHIFL', 'ACTM4', 'PHYSP', 'BORDL', 'FLAFP', 'RIPO2', 'ENTNP', 'FRACF', 'KLEFL', 'GEOSK', 'NATPD', 'GEODF', 'CUPMC', 'ECO81', 'ENTFU', 'BURVG', 'CUTAK', 'AVIPA', 'PENDC', 'STAAK', 'CORUB', 'ALISL', 'PARXL', 'SALHS', 'PEDHC', 'STRG2', 'Y

In [46]:
len(bird_hog_tree)


6

In [59]:
bird_hog_tree_address="/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/hogmapX/out4/_tree_filt_7.txt"
#"/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastoma/v3a/A/f7_2kA/_tree__msa_concatanated_filtered_row_col_0.00101.txt"
bird_hog_tree= ete3.Tree(bird_hog_tree_address)


list_node=[]
for node in bird_hog_tree.traverse(strategy="postorder"):

    if node.is_leaf() : # why ?
        list_node.append(node.name)

len(list_node)

411

In [60]:
five_repetead=set(["TAEGU","FICAL","ANAPL","CHICK","SERCA","PARMJ"])
list_node_unq=set(list_node)-five_repetead
list_node_unq


{'ACACHL',
 'ACRARU',
 'AEGBEN',
 'AEGCAU',
 'AGAROS',
 'AGEPHO',
 'AILME',
 'ALACHE',
 'ALCTOR',
 'ALELAT',
 'ALERUF',
 'ALOBEC',
 'AMAGUI',
 'ANAPLA',
 'ANAZON',
 'ANHANH',
 'ANHRUF',
 'ANSCYG',
 'ANSSEM',
 'ANTCAR',
 'ANTMIN',
 'AOTNA',
 'APAVIT',
 'APHCOE',
 'APTAUS',
 'APTFOR',
 'APTHAA',
 'APTOWE',
 'APTROW',
 'AQUCHR',
 'ARAGUA',
 'ARDKOR',
 'AREINT',
 'ASASCU',
 'ATLROG',
 'ATRCLA',
 'BALREG',
 'BALREX',
 'BARMAR',
 'BOMGAR',
 'BOVIN',
 'BRAATR',
 'BRALEP',
 'BUCABY',
 'BUCCAP',
 'BUCRHI',
 'BUPERY',
 'BURBIS',
 'CAIMOS',
 'CALANN',
 'CALBOR',
 'CALJA',
 'CALNIC',
 'CALORN',
 'CALPUG',
 'CALSQU',
 'CALVIR',
 'CALWIL',
 'CAMPRO',
 'CANLF',
 'CAPHI',
 'CARCAR',
 'CARCRI',
 'CASCAS',
 'CATAUR',
 'CATFUS',
 'CAVPO',
 'CENBEN',
 'CENUNI',
 'CEPGRY',
 'CEPORN',
 'CERAT',
 'CERBRA',
 'CERCOR',
 'CERFAM',
 'CETCET',
 'CEUAER',
 'CEYCYA',
 'CHAALE',
 'CHAFRE',
 'CHAPAP',
 'CHAPEL',
 'CHATOR',
 'CHAVOC',
 'CHILA',
 'CHIMIN',
 'CHLAEN',
 'CHLCYA',
 'CHLHAR',
 'CHLMAC',
 'CHLSB',
 'CHLVIR'

In [68]:
# bird_hog_tree.prune(list_node_unq)
bird_hog_tree.write()

'(((RHIDAH:0.00916118,CHAPAP:0.00759941)1:0.00218398,(DICMEG:0.00764643,(MYIHEB:0.00930175,((LANLUD:0.0100277,(APHCOE:0.00418719,(CORMON:0.00242676,(CORCOR:0.00159595,CORBRA:0.00301169)1:0.000693276)1:0.00236036)1:0.00371554)1:0.000713619,(STRCIN:0.00728446,(PARRAG:0.00655149,IFRKOW:0.00728625)1:5.6528e-05)1:6.8882e-05)1:6.8104e-05)0:8.1981e-05)1:7.9969e-05)1:0.00134414,(EDOCOE:0.00927021,DAPCHR:0.012291)1:0.000149208,((MOHOCH:0.00887504,(PTILEU:0.0117964,(((CALWIL:0.0104743,NOTCIN:0.0163835)1:0.000624526,(CNELOR:0.0106946,(MELVER:0.0112781,((((ANTMIN:0.0143893,(POEATR:0.00417223,(PSEHUM:0.00339711,PARMAJ:0.00069855)1:0.000804694)1:0.00734026)1:0.00172581,((ALACHE:0.0166442,PANBIA:0.0165384)1:0.00104961,(NICCHL:0.0122264,(SYLVIR:0.0139867,((HIRRUS:0.0127127,(((BRAATR:0.00700473,PYCJOC:0.00676696)1:0.00539269,((SINWEB:0.00855782,(SYLATR:0.00541378,SYLBOR:0.00456145)1:0.00584524)1:0.000586819,((STEDEN:0.00419501,(ZOSHYP:0.0023303,ZOSLAT:0.00328568)1:0.00242008)1:0.00711097,(PORRUF:0.0113

In [69]:
ncbi_sub_tree_small.write()

'((((((BUCCAP:1,GALDEA:1)1:1,((((TAEGUT:1,LONSTR:1)1:1,(VIDCHA:1,VIDMAC:1)1:1)1:1,(((LOXLEU:1,LOXCUR:1)1:1,SERCAN:1)1:1,((CALORN:1,EMBFUC:1)1:1,(CARCAR:1,PASAMO:1,PHEMEL:1)1:1)1:1,UROPYL:1,CHLVIR:1,PEUTAE:1,HEMWIL:1)1:1,((PRUFUL:1,PRUHIM:1)1:1,PASDOM:1)1:1,(PROCAF:1,LEPASP:1)1:1,(AGEPHO:1,QUIMEX:1,MOLATE:1)1:1,(SETCOR:1,SETKIR:1)1:1,OREARF:1,MELVER:1,MOTALB:1,DICEXI:1,PLONIG:1)1:1,(((ZOSLAT:1,ZOSHYP:1)1:1,HYPCIN:1,STEDEN:1)1:1,(PORRUF:1,ILLCLE:1,PTEMEL:1,ERPZAN:1,POSRUF:1,MYSCRO:1,OXYMAD:1)1:1,(BRAATR:1,PYCJOC:1,NICCHL:1)1:1,((ACRARU:1,CETCET:1,HIPICT:1,SYLVIR:1,HYLPRA:1)1:1,(PANBIA:1,REGSAT:1)1:1,(SYLATR:1,SYLBOR:1)1:1,SINWEB:1)1:1,(PHYTRO:1,RHASIB:1)1:1,HIRRUS:1,ALACHE:1,LOCOCH:1,HORVUL:1,CISJUN:1,LEILUT:1,AEGCAU:1)1:1,(((CHLCYA:1,CHLHAR:1)1:1,IRECYA:1)1:1,((CORMON:1,CORCOR:1,CORBRA:1)1:1,CNELOR:1,APHCOE:1,ORIORI:1)1:1,(ALERUF:1,EULNIG:1,RHALEU:1,DAPCHR:1,FALFRO:1,PACPHI:1)1:1,(IFRKOW:1,PTILEU:1)1:1,(ERYMCC:1,DICMEG:1,MACNIG:1)1:1,LANLUD:1,MYIHEB:1,CALWIL:1,DRYGAM:1,EDOCOE:1,PARRAG:1

In [64]:
len(bird_hog_tree)

405

In [66]:
len(ncbi_sub_tree_small)

405

In [ ]:
reroot_at_edge



In [71]:
bird_hog_tree.rerootatedge("XENLA")

# reroot_at_edge

AttributeError: 'TreeNode' object has no attribute 'rerootatedge'

In [67]:
out = bird_hog_tree.robinson_foulds(ncbi_sub_tree_small,unrooted_trees=True)  # , expand_polytomies = True
(rf, rf_max, common_attrs, names, edges_t1, edges_t2, discarded_edges_t1) =out

# rf, max_rf, common_leaves, parts_t1, parts_t2 = t1.robinson_foulds(t2)


print("RF distance is %s over a total of %s" %(rf, rf_max))


RF distance is 345 over a total of 569


In [91]:
#bird_hog_tree, ncbi_sub_tree_small
len(bird_hog_tree)

405

In [94]:
len(out[2]),len(out[3]),len(out[4]),len(out[5])

(405, 808, 573, 0)

In [88]:
out[0],out[1],list(out[2])[:3],list(out[3])[:3],list(out[4])[:3]

(345,
 569,
 ['CINMEX', 'SITEUR', 'PANPA'],
 [(('ACACHL',
    'ACRARU',
    'AEGBEN',
    'AEGCAU',
    'AGAROS',
    'AGEPHO',
    'AILME',
    'ALACHE',
    'ALCTOR',
    'ALELAT',
    'ALERUF',
    'ALOBEC',
    'AMAGUI',
    'ANAPLA',
    'ANAZON',
    'ANHANH',
    'ANHRUF',
    'ANSCYG',
    'ANSSEM',
    'ANTCAR',
    'ANTMIN',
    'AOTNA',
    'APAVIT',
    'APHCOE',
    'APTAUS',
    'APTFOR',
    'APTHAA',
    'APTOWE',
    'APTROW',
    'AQUCHR',
    'ARAGUA',
    'ARDKOR',
    'AREINT',
    'ASASCU',
    'ATLROG',
    'ATRCLA',
    'BALREG',
    'BALREX',
    'BARMAR',
    'BOMGAR',
    'BOVIN',
    'BRAATR',
    'BRALEP',
    'BUCABY',
    'BUCCAP',
    'BUCRHI',
    'BUPERY',
    'BURBIS',
    'CAIMOS',
    'CALANN',
    'CALBOR',
    'CALJA',
    'CALNIC',
    'CALORN',
    'CALPUG',
    'CALSQU',
    'CALVIR',
    'CALWIL',
    'CAMPRO',
    'CANLF',
    'CAPHI',
    'CARCAR',
    'CARCRI',
    'CASCAS',
    'CATAUR',
    'CATFUS',
    'CAVPO',
    'CENBEN',
    'CENUNI

In [96]:
ete3.v

<module 'ete3.version' from '/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-packages/ete3/version.py'>